# Day 22

## Read input

In [51]:
import re
from collections import namedtuple

from utils import read_input

Point = namedtuple('Point', ['x', 'y', 'z'])
Instruction = namedtuple('Instruction', ['action', 'start', 'end'])

def transformer(line):
    m = re.match(r'(?P<action>on|off) x=(?P<x>-?\d+..-?\d+),y=(?P<y>-?\d+..-?\d+),z=(?P<z>-?\d+..-?\d+)', line)
    action = m.group('action')
    x_range = m.group('x')
    y_range = m.group('y')
    z_range = m.group('z')
    
    x1, x2 = x_range.split('..')
    y1, y2 = y_range.split('..')
    z1, z2 = z_range.split('..')
    
    start = Point(x=int(x1), y=int(y1), z=int(z1))
    end = Point(x=int(x2), y=int(y2), z=int(z2))
    
    return Instruction(action, start, end)
    
instructions = read_input(22, transformer, example=True)

In [10]:
# Debug stuff

all_starts = [i.start for i in instructions]
all_ends = [i.end for i in instructions]
all_points = all_starts + all_ends


print("X:")
print(min(p.x for p in all_points))
print(max(p.x for p in all_points))

print("Y:")
print(min(p.y for p in all_points))
print(max(p.y for p in all_points))

print("Z:")
print(min(p.z for p in all_points))
print(max(p.z for p in all_points))

X:
-97005
96577
Y:
-96164
97145
Z:
-96514
96151


## Part 1

In [3]:
def get_cuboids_in_range(start, end):
    cuboids = []
    if start.x < -50 and end.x < -50 or start.x > 50 and end.x > 50:
        return []
    if start.y < -50 and end.y < -50 or start.y > 50 and end.y > 50:
        return []
    if start.z < -50 and end.z < - 50 or start.z > 50 and end.z > 50:
        return []

    min_x = max(-50, min(50, start.x))
    max_x = min(50, max(-50, end.x))
    min_y = max(-50, min(50, start.y))
    max_y = min(50, max(-50, end.y))
    min_z = max(-50, min(50, start.z))
    max_z = min(50, max(-50, end.z))
    
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            for z in range(min_z, max_z + 1):
                cuboids.append(Point(x,y,z))
    return cuboids

I learned today that `set` has two methods for removing items: `discard` and `remove`. I've been using `remove` with an `if` guard all these years but there's a nice difference:

> discard(...) method of builtins.set instance
>    Remove an element from a set if it is a member.
>
>    If the element is not a member, do nothing.

>remove(...) method of builtins.set instance
>    Remove an element from a set; it must be a member.
>
>    If the element is not a member, raise a KeyError.

So whenever you need to remove items that might not exist in a set, `discard` is a good option.

In [32]:
def activate_cuboids(instructions):
    lit = set()

    for instruction in instructions:
        for point in get_cuboids_in_range(instruction.start, instruction.end):
            match instruction.action:
                case 'on':
                    lit.add(point)
                case 'off':
                    lit.discard(point)
    return lit

In [33]:
result = len(activate_cuboids(instructions))
print(f'Solution: {result}')
assert result == 648681

Solution: 648681


## Part 2

In [61]:
def does_line_intersect(one_start, one_end, other_start, other_end):
    return (
        one_start < other_start < one_end or
        one_start < other_end < one_end or
        other_start < one_start < other_end or
        other_start < one_end < other_end
    )

def get_line_intersection(one_start, one_end, other_start, other_end):
    intersection_start = max(one_start, other_start)
    intersection_end = max(one_end, other_end)
    
    return (intersection_start, intersection_end)
    
class Area:
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.off = []
        
    def is_overlapping(self, other):
        return (
            does_line_intersect(self.start.x, self.end.x, other.start.x, other.end.x) and
            does_line_intersect(self.start.y, self.end.y, other.start.y, other.end.y) and
            does_line_intersect(self.start.z, self.end.z, other.start.z, other.end.z)
        )
        
    def substraction(self, other):
        if self.is_overlapping(other):
            x = get_line_intersection(self.start.x, self.end.x, other.start.x, other.end.x)
            y = get_line_intersection(self.start.y, self.end.y, other.start.y, other.end.y)
            z = get_line_intersection(self.start.z, self.end.z, other.start.z, other.end.z)
            
            for o in self.off:
                o.substraction(other)
            
            self.off.append(Area(Point(x[0], y[0], z[0]), end=Point(x[1], y[1], z[1])))
            
    def volume(self):
        full_volume = (self.end.x - self.start.x + 1) * (self.end.y - self.start.y + 1) * (self.end.z - self.start.z + 1)
        off_volume = sum(area.volume() for area in self.off)        
        return full_volume - off_volume

In [62]:
areas = []

for instruction in instructions:
    area = Area(instruction.start, instruction.end)
    
    for a in areas:
        a.substraction(area)
        
    if instruction.action == 'on':
        areas.append(area)
        
print('Our', sum(area.volume() for area in areas))
print('Goal', 2758514936282235)

Our 575818452252378
Goal 2758514936282235
